In [1]:
%matplotlib inline
import pandas as pd
import pylab
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sci
from sklearn.neighbors import KNeighborsClassifier
from sklearn import cross_validation
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.model_selection import KFold
from sklearn.linear_model import Ridge
from sklearn import linear_model
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn import tree
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder

from numpy import array
from numpy import argmax
from keras.utils import to_categorical
from sklearn.ensemble import RandomForestRegressor

C:\Users\Max\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using TensorFlow backend.


In [2]:
df = pd.read_csv('C:/Users/Max/Downloads/data/train_women_processed.csv')

In [13]:


def rmsle(y, y_pred):
    assert len(y) == len(y_pred)
    to_sum = [(np.log(y_pred[i] + 1) - np.log(y[i] + 1)) ** 2.0 for i,pred in enumerate(y_pred)]
    return (sum(to_sum) * (1.0/len(y))) ** 0.5

In [28]:
def rmse(y, y_pred):
    assert len(y) == len(y_pred)
    to_sum = [(y_pred[i]-y[i]) ** 2.0 for i,pred in enumerate(y_pred)]
    return (sum(to_sum) * (1.0/len(y))) ** 0.5

In [5]:
"""dictionary=df[['brand_name','price']].groupby('brand_name').mean()
di=dictionary.to_dict()['price']
df['brand_name']=df.loc[:100000,'brand_name'].replace(di)"""

"dictionary=df[['brand_name','price']].groupby('brand_name').mean()\ndi=dictionary.to_dict()['price']\ndf['brand_name']=df.loc[:100000,'brand_name'].replace(di)"

In [6]:
df.drop('train_id',axis=1,inplace=True)
df.loc[:, df.dtypes == np.int64]=df.loc[:, df.dtypes == np.int64].astype('int16')

In [7]:
df[['category2','category3','brand_name']]= df[['category2','category3','brand_name']].astype('category')
df_x=df.drop(['name','item_description','category_name','category','category2','category3' ,'price','brand_name'],axis=1)
df_y=df['price'].values

In [8]:
#creating one hot encoding array for category2 and category3
le = preprocessing.LabelEncoder()
data = le.fit_transform(df.category2)
data = array(data)

category2 = to_categorical(data)
# invert encoding
#inverted = argmax(encoded[0])
#print(inverted)
#print(type(encoded))
data = le.fit_transform(df.category3)
data = array(data)
category3 = to_categorical(data)

In [9]:
df_x=np.concatenate([df_x.values,category2],axis=1)

In [31]:
i=0
df2 = pd.DataFrame(np.zeros((1,10)) ) 
for k in range(5):
    kf = KFold(n_splits=4)
    for train_index, test_index in kf.split(df):
        result=[]
        X_train, X_test = df_x[train_index], df_x[test_index]
        y_train, y_test = df_y[train_index], df_y[test_index]

        neigh = linear_model.Lasso(alpha=0.05*(k+1))
        neigh.fit(X_train,y_train)
        err=rmse(neigh.predict(X_test),y_test)
        result.append(err)
        
        
    df2.iloc[0,k]=np.average(result)
print(df2)

37.662823955580706
37.43760293588879
36.86345688161297
37.533079104009985
37.7355554390166
37.51227529884018
36.940089786095974
37.60544285864586
37.81994843585145
37.597873296308855
37.02718045593601
37.68652203577686
37.89527262899069
37.67601149740581
37.1053702003642
37.76098879389184
37.972358839840886
37.75500611758901
37.18365699676178
37.83366822642087
           0          1          2          3          4    5    6    7    8  \
0   0.000000   0.000000   0.000000   0.000000   0.000000  0.0  0.0  0.0  0.0   
1  37.533079  37.605443  37.686522  37.760989  37.833668  0.0  0.0  0.0  0.0   

     9  
0  0.0  
1  0.0  


In [ ]:
i=0
df2 = pd.DataFrame(np.zeros((1,10)) ) 
for k in range(5):
    kf = KFold(n_splits=4)
    for train_index, test_index in kf.split(df):
        result=[]
        X_train, X_test = df_x[train_index], df_x[test_index]
        y_train, y_test = df_y[train_index], df_y[test_index]

        neigh = linear_model.Lasso(alpha=0.05*(k+1))
        neigh.fit(X_train,y_train)
        err=rmse(neigh.predict(X_test),y_test)
        result.append(err)
        
        
    df2.iloc[0,k]=np.average(result)
print(df2)

In [ ]:
i=0
df2 = pd.DataFrame(np.zeros((5,5)) )
hidden_layer_sizes=[(50,)(100,),(50,50),(100,50) (50,50,50)]
for i in range(5):
    for k in range(5):
        kf = KFold(n_splits=4)
        for train_index, test_index in kf.split(df):
            result=[]
            X_train, X_test = df_x[train_index], df_x[test_index]
            y_train, y_test = df_y[train_index], df_y[test_index]

            neigh = linear_model.sklearn.neural_network.MLPClassifier(alpha=0.05*(k+1),hidden_layer_sizes=hidden_layer_sizes[i])
            neigh.fit(X_train,y_train)
            err=rmse(neigh.predict(X_test),y_test)
            result.append(err)


        df2.iloc[i,k]=np.average(result)
print(df2)



In [ ]:
n_estimators=[30,50, 100, 150, 200]
max_depth=[10, 20, 40, 70, None]
i=0
df2 = pd.DataFrame(np.zeros((5,10)) ) 
for i in range(5):
    for k in range(5):
        kf = KFold(n_splits=4)
        for train_index, test_index in kf.split(df):
            result=[]
            X_train, X_test = df_x[train_index], df_x[test_index]
            y_train, y_test = df_y[train_index], df_y[test_index]

            neigh = RandomForestRegressor(random_state = 42,n_estimators=n_estimators[i],max_depth=max_depth[k])
            neigh.fit(X_train,y_train)
            err=rmse(neigh.predict(X_test),y_test)
            result.append(err)
        
        
    df2.iloc[i,k]=np.average(result)
print(df2)